<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_1_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
bash: line 1: syntax error near unexpected token `newline'
100  9115  100  9115    0     0  48227      0 --:--:-- --:--:-- --:--:-- 48227
bash: line 1: `<!DOCTYPE html>'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [14]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [15]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [18]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [21]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [22]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.216046,4.650535,0.249191,16:13


In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.481661,4.311352,0.282952,16:12


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.161139,4.271493,0.287249,16:14


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.109604,4.240792,0.289844,16:14


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.825126,3.780637,0.393462,01:06


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.741907,3.207452,0.460048,01:20


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.280173,2.941175,0.498789,03:18


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.031142,2.684063,0.538741,03:24
1,1.866049,2.542806,0.550847,03:23


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.711639,2.401791,0.576271,03:18
1,1.579971,2.337824,0.577482,03:21


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.507094,2.240454,0.600484,03:29
1,1.415134,2.208860,0.610169,03:21


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.356133,2.166026,0.608959,03:16
1,1.293186,2.118217,0.617433,03:19


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.261973,2.117191,0.625908,03:32
1,1.158309,2.114976,0.631961,03:23


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.164217,2.088867,0.642857,03:33
1,1.164335,2.074365,0.635593,03:28
2,1.061183,2.009067,0.631961,03:13
3,0.968861,2.010116,0.638015,03:32


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.989476,2.062728,0.635593,03:21
1,1.023472,2.068565,0.644068,03:27
2,0.940271,1.991791,0.653753,03:26
3,0.876584,1.988496,0.651332,03:14


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.930677,2.029230,0.653753,03:17
1,0.969901,2.090549,0.648910,03:18
2,0.910992,2.049645,0.653753,03:13
3,0.811332,2.035345,0.654964,03:16


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.864274,2.071159,0.642857,03:14
1,0.913817,2.128101,0.652542,03:34
2,0.871804,2.103888,0.648910,03:32
3,0.822832,2.115309,0.653753,03:19


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.806319,2.141736,0.652542,03:31
1,0.917991,2.164828,0.648910,03:30
2,0.918782,2.113229,0.648910,03:15
3,0.793010,2.059775,0.658596,03:14


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.830269,2.129710,0.651332,03:18
1,0.823856,2.063061,0.652542,03:13
2,0.824458,2.094099,0.664649,03:15
3,0.793820,2.109341,0.654964,03:22


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.787275,2.118257,0.661017,03:32
1,0.817255,2.118676,0.664649,03:14
2,0.827666,2.142518,0.659806,03:28
3,0.809057,2.117676,0.654964,03:19


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.726842,2.085145,0.658596,03:16
1,0.803317,2.175801,0.656174,03:25
2,0.767097,2.185637,0.659806,03:21
3,0.737360,2.150725,0.659806,03:18


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.758371,2.182659,0.657385,03:24
1,0.794968,2.114074,0.663438,03:26
2,0.718503,2.111192,0.651332,03:20
3,0.771359,2.129795,0.662228,03:30


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.806152,2.202622,0.657385,03:27
1,0.792743,2.129685,0.661017,03:28


In [46]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.754485,2.154877,0.653753,03:23
1,0.731420,2.148532,0.661017,03:19


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[260, 2, 8, 301, 8, 8, 301, 16, 20, 830, 933, 84, 27, 31, 830, 37, 37, 37, 37, 37, 129, 524, 289, 289, 37, 260, 95, 360, 44, 813, 325, 49, 475, 100, 100, 100, 100, 49, 53, 53, 785, 260, 60, 61, 62, 830, 70, 343, 71, 41, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 686, 830, 830, 830, 830, 830, 289, 281, 289, 80, 9, 80, 80, 80, 80, 491, 260, 108, 89, 214, 92, 92, 92, 94, 94, 94, 94, 94, 94, 431, 431, 431, 94, 96, 96, 518, 234, 100, 100, 100, 100, 766, 100, 100, 572, 100, 100, 100, 358, 103, 830, 105, 107, 877, 108, 108, 190, 205, 462, 111, 111, 111, 111, 111, 129, 111, 830, 117, 120, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 802, 260, 321, 73, 455, 135, 94, 139, 896, 455, 447, 529, 864, 289, 158, 166, 830, 770, 830, 171, 686, 289, 211, 356, 289, 830, 830, 830, 492, 809, 524, 2, 794, 182, 994, 994, 182, 182, 182, 289, 211, 183, 183, 183, 183, 183, 183, 183, 8

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

471
0.5432525951557093
